In [1]:
# Copyright (c) 2024 Byeonghyeon Kim 
# github site: https://github.com/bhkim003/ByeonghyeonKim
# email: bhkim003@snu.ac.kr
 
# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:
 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
# FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
# COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
# IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [2]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import time

from snntorch import spikegen
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML

from tqdm import tqdm

from apex.parallel import DistributedDataParallel as DDP

import random

In [3]:
# my module import
from modules import *

# modules 폴더에 새모듈.py 만들면
# modules/__init__py 파일에 form .새모듈 import * 하셈
# 그리고 새모듈.py에서 from modules.새모듈 import * 하셈


In [4]:
def my_snn_system(devices = "0,1,2,3",
                    my_seed = 42,
                    TIME = 8,
                    BATCH = 256,
                    IMAGE_SIZE = 32,
                    which_data = 'CIFAR10',
                    CLASS_NUM = 10,
                    data_path = '/data2',
                    rate_coding = True,
    
                    lif_layer_v_init = 0.0,
                    lif_layer_v_decay = 0.6,
                    lif_layer_v_threshold = 1.2,
                    lif_layer_v_reset = 0.0,
                    lif_layer_sg_width = 1,

                    # synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
                    synapse_conv_kernel_size = 3,
                    synapse_conv_stride = 1,
                    synapse_conv_padding = 1,
                    synapse_conv_trace_const1 = 1,
                    synapse_conv_trace_const2 = 0.6,

                    # synapse_fc_out_features = CLASS_NUM,
                    synapse_fc_trace_const1 = 1,
                    synapse_fc_trace_const2 = 0.6,

                    pre_trained = False,
                    convTrue_fcFalse = True,
                    cfg = [64, 64],
                    net_print = False, # True # False
                    pre_trained_path = "net_save/save_now_net.pth",
                    learning_rate = 0.0001,
                    epoch_num = 200,
                    verbose_interval = 100, #숫자 크게 하면 꺼짐
                    validation_interval = 10, #숫자 크게 하면 꺼짐
                    tdBN_on = False,
                    BN_on = False,

                    surrogate = 'sigmoid',

                    gradient_verbose = False,

                    BPTT_on = False,

                    scheduler_name = 'no',
                    
                    ddp_on = True,

                    nda_net = False,
                  ):


    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
    os.environ["CUDA_VISIBLE_DEVICES"]= devices

    
    torch.manual_seed(my_seed)


    
    # data loader, pixel channel, class num
    train_loader, test_loader, synapse_conv_in_channels = data_loader(
            which_data,
            data_path, 
            rate_coding, 
            BATCH, 
            IMAGE_SIZE,
            ddp_on)
    synapse_fc_out_features = CLASS_NUM


    ## parameter number calculator ##########################################
    params_num = 0
    img_size = IMAGE_SIZE 
    bias_param = 1 # 1 or 0
    classifier_making = False
    if (convTrue_fcFalse == True):
        past_kernel = synapse_conv_in_channels
        for kernel in cfg:
            if (classifier_making == False):
                if (type(kernel) == list):
                    for residual_kernel in kernel:
                        params_num += residual_kernel * ((synapse_conv_kernel_size**2) * past_kernel + bias_param)
                        past_kernel = residual_kernel
                elif (kernel == 'P' or kernel == 'M'):
                    img_size = img_size // 2
                elif (kernel == 'L'):
                    classifier_making = True
                    past_kernel = past_kernel * (img_size**2)
                else:
                    params_num += kernel * (synapse_conv_kernel_size**2 * past_kernel + bias_param)
                    past_kernel = kernel    
            else: # classifier making
                params_num += (past_kernel + bias_param) * kernel
                past_kernel = kernel
        
        
        if classifier_making == False:
            past_kernel = past_kernel*img_size*img_size

        params_num += (past_kernel + bias_param) * synapse_fc_out_features
    else:
        past_in_channel = synapse_conv_in_channels*img_size*img_size
        for in_channel in cfg:
            if (type(in_channel) == list):
                for residual_in_channel in in_channel:
                    params_num += (past_in_channel + bias_param) * residual_in_channel
                    past_in_channel = residual_in_channel
            # elif (in_channel == 'M'): #it's a holy FC layer!
            #     img_size = img_size // 2
            else:
                params_num += (past_in_channel + bias_param) * in_channel
                past_in_channel = in_channel
        params_num += (past_in_channel + bias_param) * synapse_fc_out_features
    ## parameter number calculator ##########################################


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if pre_trained == False:
        if (convTrue_fcFalse == False):
            net = MY_SNN_FC(cfg, synapse_conv_in_channels, IMAGE_SIZE, synapse_fc_out_features,
                     synapse_fc_trace_const1, synapse_fc_trace_const2, 
                     lif_layer_v_init, lif_layer_v_decay, 
                     lif_layer_v_threshold, lif_layer_v_reset,
                     lif_layer_sg_width,
                     tdBN_on,
                     BN_on, TIME,
                     surrogate,
                     BPTT_on).to(device)
        else:
            net = MY_SNN_CONV(cfg, synapse_conv_in_channels, IMAGE_SIZE,
                     synapse_conv_kernel_size, synapse_conv_stride, 
                     synapse_conv_padding, synapse_conv_trace_const1, 
                     synapse_conv_trace_const2, 
                     lif_layer_v_init, lif_layer_v_decay, 
                     lif_layer_v_threshold, lif_layer_v_reset,
                     lif_layer_sg_width,
                     synapse_fc_out_features, synapse_fc_trace_const1, synapse_fc_trace_const2,
                     tdBN_on,
                     BN_on, TIME,
                     surrogate,
                     BPTT_on).to(device)
        
        if (nda_net == True):
            net = VGG(cfg = cfg, num_classes=10, batch_norm = tdBN_on, in_c = synapse_conv_in_channels, 
                      lif_layer_v_threshold=lif_layer_v_threshold, lif_layer_v_decay=lif_layer_v_decay, lif_layer_sg_width=lif_layer_sg_width)
            net.T = TIME
        net = torch.nn.DataParallel(net)
    else:
        net = torch.load(pre_trained_path)


    net = net.to(device)
    if (net_print == True):
        print(net)
    
    ## param num and memory estimation except BN at MY calculation ##########################################
    real_param_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
    # Batch norm 있으면 아래 두 개 서로 다를 수 있음.
    # assert real_param_num == params_num, f'parameter number is not same. real_param_num: {real_param_num}, params_num: {params_num}'    
    print('='*50)
    print(f"My Num of PARAMS: {params_num:,}, system's param_num : {real_param_num:,}")
    memory = params_num / 8 / 1024 / 1024 # MB
    precision = 32
    memory = memory * precision 
    print(f"Memory: {memory:.2f}MiB at {precision}-bit")
    print('='*50)
    ##########################################################################

    criterion = nn.CrossEntropyLoss().to(device)
    # optimizer = torch.optim.Adam(net.parameters(), lr=0.00001)
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate/256 * BATCH, weight_decay=1e-4)


    if (scheduler_name == 'StepLR'):
        scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    elif (scheduler_name == 'ExponentialLR'):
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
    elif (scheduler_name == 'ReduceLROnPlateau'):
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
    elif (scheduler_name == 'CosineAnnealingLR'):
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, eta_min=0, T_max=50)
    elif (scheduler_name == 'OneCycleLR'):
        scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_loader), epochs=100)
    else:
        pass # 'no' scheduler

    val_acc = 0
    val_acc_now = 0
    elapsed_time_val = 0
    for epoch in range(epoch_num):
        print('EPOCH', epoch)
        epoch_start_time = time.time()
        running_loss = 0.0
        
        iterator = enumerate(train_loader, 0)
        if (ddp_on == True):
            if torch.distributed.get_rank() == 0:   
                iterator = tqdm(iterator, total=len(train_loader), desc='train', dynamic_ncols=True, position=0, leave=True)
        else:
            iterator = tqdm(iterator, total=len(train_loader), desc='train', dynamic_ncols=True, position=0, leave=True)

        for i, data in iterator:
            iter_one_train_time_start = time.time()
            net.train()

            ## data loading #################################
            inputs, labels = data

            
            if (which_data == 'DVS-CIFAR10'):
                inputs = inputs.permute(1, 0, 2, 3, 4)
            elif rate_coding == True :
                inputs = spikegen.rate(inputs, num_steps=TIME)
            else :
                inputs = inputs.repeat(TIME, 1, 1, 1, 1)
            # inputs: [Time, Batch, Channel, Height, Width]  
            ################################################# 

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # inputs: [Time, Batch, Channel, Height, Width]   
            inputs = inputs.permute(1, 0, 2, 3, 4) # net에 넣어줄때는 batch가 젤 앞 차원으로 와야함. # dataparallel때매
            # inputs: [Batch, Time, Channel, Height, Width]   
        
            outputs = net(inputs)

            batch = BATCH 
            if labels.size(0) != BATCH: 
                batch = labels.size(0)

            


            ####### training accruacy print ###############################
            correct = 0
            total = 0
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted[0:batch] == labels).sum().item()
            if i % verbose_interval == verbose_interval-1:
                print(f'{epoch}-{i} training acc: {100 * correct / total:.2f}%, lr={[f"{lr}" for lr in (param_group["lr"] for param_group in optimizer.param_groups)]}, val_acc: {100 * val_acc_now:.2f}%')
            training_acc_string = f'{epoch}-{i}/{len(train_loader)} tr_acc: {100 * correct / total:.2f}%, lr={[f"{lr}" for lr in (param_group["lr"] for param_group in optimizer.param_groups)]}'
            
            ################################################################
            

            loss = criterion(outputs[0:batch,:], labels)
            loss.backward()


            ### gradinet verbose ##########################################
            if (gradient_verbose == True):
                if (i % verbose_interval == verbose_interval-1):
                    print('\n\nepoch', epoch, 'iter', i)
                    for name, param in net.named_parameters():
                        if param.requires_grad:
                            print('\n\n\n\n' , name, param.grad)
            ################################################################
            
            optimizer.step()

            running_loss += loss.item()
            # print("Epoch: {}, Iter: {}, Loss: {}".format(epoch + 1, i + 1, running_loss / 100))

            iter_one_train_time_end = time.time()
            elapsed_time = iter_one_train_time_end - iter_one_train_time_start  # 실행 시간 계산

            if (i % verbose_interval == verbose_interval-1):
                print(f"iter_one_train_time: {elapsed_time} seconds, last one_val_time: {elapsed_time_val} seconds\n")

            ##### validation ##############################################
            if i % validation_interval == validation_interval-1:
                iter_one_val_time_start = time.time()
                
                correct = 0
                total = 0
                with torch.no_grad():
                    net.eval()
                    for data in test_loader:
                        ## data loading #################################
                        inputs, labels = data

                        
                        if (which_data == 'DVS-CIFAR10'):
                            inputs = inputs.permute(1, 0, 2, 3, 4)
                        elif rate_coding == True :
                            inputs = spikegen.rate(inputs, num_steps=TIME)
                        else :
                            inputs = inputs.repeat(TIME, 1, 1, 1, 1)
                        # inputs: [Time, Batch, Channel, Height, Width]  
                        ################################################# 

                        inputs = inputs.to(device)
                        labels = labels.to(device)
                        outputs = net(inputs.permute(1, 0, 2, 3, 4))
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        batch = BATCH 
                        if labels.size(0) != BATCH: 
                            batch = labels.size(0)
                        correct += (predicted[0:batch] == labels).sum().item()
                        val_loss = criterion(outputs[0:batch,:], labels)

                    val_acc_now = correct / total
                    # print(f'{epoch}-{i} validation acc: {100 * val_acc_now:.2f}%, lr={[f"{lr:.10f}" for lr in (param_group["lr"] for param_group in optimizer.param_groups)]}')

                iter_one_val_time_end = time.time()
                elapsed_time_val = iter_one_val_time_end - iter_one_val_time_start  # 실행 시간 계산
                # print(f"iter_one_val_time: {elapsed_time_val} seconds")

                # network save
                if val_acc < val_acc_now:
                    val_acc = val_acc_now
                    torch.save(net.state_dict(), "net_save/save_now_net_weights.pth")
                    torch.save(net, "net_save/save_now_net.pth")
                    torch.save(net.module.state_dict(), "net_save/save_now_net_weights2.pth")
                    torch.save(net.module, "net_save/save_now_net2.pth")
            ################################################################
            iterator.set_description(f"train: {training_acc_string}, tr_loss: {loss}, val_acc: {100 * val_acc_now:.2f}%")     
        if (scheduler_name != 'no'):
            if (scheduler_name == 'ReduceLROnPlateau'):
                scheduler.step(val_loss)
            else:
                scheduler.step()

        
        
        epoch_time_end = time.time()
        epoch_time = epoch_time_end - epoch_start_time  # 실행 시간 계산
        
        print(f"epoch_time: {epoch_time} seconds")
        print('\n')


In [5]:
### my_snn control board ########################
decay = 0.25 # 0.875 0.25 0.125 0.75 0.5

my_snn_system(  devices = "0,1,2,3,4,5",
                my_seed = 42,
                TIME = 10, # dvscifar 10
                BATCH = 128, # batch norm 할거면 2이상으로 해야함
                IMAGE_SIZE = 48, # dvscifar 48 # MNIST 28 # CIFAR10 32
                which_data = 'DVS-CIFAR10',# 'CIFAR10' 'MNIST' 'FASHION_MNIST' 'DVS-CIFAR10'
                CLASS_NUM = 10,
                data_path = '/data2', # YOU NEED TO CHANGE THIS
                rate_coding = False, # True # False

                lif_layer_v_init = 0.0,
                lif_layer_v_decay = decay,
                lif_layer_v_threshold = 0.5,
                lif_layer_v_reset = 0.0, #현재 안씀. 걍 빼기 해버림
                lif_layer_sg_width = 1.0, # surrogate sigmoid 쓸 때는 의미없음

                # synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
                synapse_conv_kernel_size = 3,
                synapse_conv_stride = 1,
                synapse_conv_padding = 1,
                synapse_conv_trace_const1 = 1,
                synapse_conv_trace_const2 = decay, # lif_layer_v_decay

                # synapse_fc_out_features = CLASS_NUM,
                synapse_fc_trace_const1 = 1,
                synapse_fc_trace_const2 = decay, # lif_layer_v_decay

                pre_trained = False, # True # False
                convTrue_fcFalse = True, # True # False

                # 'P' for average pooling, 'M' for maxpooling, 'L' for linear classifier, [  ] for residual block
                # cfg = [64],
                # cfg = [64,[64,64],64], # 끝에 linear classifier 하나 자동으로 붙습니다
                # cfg = [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
                # cfg = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512],
                cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512],
                # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512, 'L', 4096, 4096],


                net_print = False, # True # False
                pre_trained_path = "net_save/save_now_net.pth",
                learning_rate = 0.001,
                epoch_num = 200,
                verbose_interval = 50, #숫자 크게 하면 꺼짐
                validation_interval = 50, #숫자 크게 하면 꺼짐
                tdBN_on = True,  # True # False
                BN_on = False,  # True # False
                
                surrogate = 'rough_rectangle', # 'rectangle' 'sigmoid' 'rough_rectangle'
                
                gradient_verbose = False,  # True # False  # weight gradient 각 layer마다 띄워줌

                BPTT_on = True,  # True # False # True이면 BPTT, False이면 OTTT

                scheduler_name = 'no', # 'no' 'StepLR' 'ExponentialLR' 'ReduceLROnPlateau' 'CosineAnnealingLR' 'OneCycleLR'
                
                ddp_on = False,   # True # False

                nda_net = False,   # True # False
                ) 
# sigmoid와 BN이 있어야 잘된다.
# average pooling이 낫다.

# nda에서는 decay = 0.25, threshold = 0.5, width =1, surrogate = rectangle
## OTTT 에서는 decay = 0.5, threshold = 1.0, surrogate = sigmoid


My Num of PARAMS: 9,404,234, system's param_num : 9,409,738
Memory: 35.87MiB at 32-bit
EPOCH 0


train: 0-48/71 tr_acc: 25.78%, lr=['0.0005'], tr_loss: 2.052292823791504, val_acc: 0.00%:  69%|██████▉   | 49/71 [00:35<00:08,  2.63it/s] 

0-49 training acc: 21.88%, lr=['0.0005'], val_acc: 0.00%
iter_one_train_time: 0.36029720306396484 seconds, last one_val_time: 0 seconds



train: 0-70/71 tr_acc: 37.50%, lr=['0.0005'], tr_loss: 1.903556227684021, val_acc: 25.60%: 100%|██████████| 71/71 [00:45<00:00,  1.57it/s] 

epoch_time: 45.47909212112427 seconds


EPOCH 1


...


train: 10-48/71 tr_acc: 62.50%, lr=['0.0005'], tr_loss: 1.1049621105194092, val_acc: 49.90%:  69%|██████▉   | 49/71 [00:18<00:07,  2.84it/s]

10-49 training acc: 64.06%, lr=['0.0005'], val_acc: 49.90%
iter_one_train_time: 0.4488356113433838 seconds, last one_val_time: 1.7718403339385986 seconds



train: 10-70/71 tr_acc: 55.00%, lr=['0.0005'], tr_loss: 1.3147170543670654, val_acc: 52.20%: 100%|██████████| 71/71 [00:27<00:00,  2.54it/s]

epoch_time: 28.213001489639282 seconds


EPOCH 11



train: 11-25/71 tr_acc: 64.06%, lr=['0.0005'], tr_loss: 1.095795750617981, val_acc: 52.20%:  37%|███▋      | 26/71 [00:09<00:16,  2.75it/s] 

In [ ]:
# my_snn_system(  devices = "2,3,4,5",
#                 my_seed = 42,
#                 TIME = 8,
#                 BATCH = 128, # batch norm 할거면 2이상으로 해야함
#                 IMAGE_SIZE = 32,
#                 which_data = 'CIFAR10',# 'CIFAR10' 'MNIST' 'FASHION_MNIST' 'DVS-CIFAR10'
#                 CLASS_NUM = 10,
#                 data_path = '/data2', # YOU NEED TO CHANGE THIS
#                 rate_coding = False, # True # False

#                 lif_layer_v_init = 0.0,
#                 lif_layer_v_decay = decay,
#                 lif_layer_v_threshold = 1.2,
#                 lif_layer_v_reset = 0.0, #현재 안씀. 걍 빼기 해버림
#                 lif_layer_sg_width = 1.0, # surrogate sigmoid 쓸 때는 의미없음

#                 # synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
#                 synapse_conv_kernel_size = 3,
#                 synapse_conv_stride = 1,
#                 synapse_conv_padding = 1,
#                 synapse_conv_trace_const1 = 1,
#                 synapse_conv_trace_const2 = decay, # lif_layer_v_decay

#                 # synapse_fc_out_features = CLASS_NUM,
#                 synapse_fc_trace_const1 = 1,
#                 synapse_fc_trace_const2 = decay, # lif_layer_v_decay

#                 pre_trained = False, # True # False
#                 convTrue_fcFalse = True, # True # False
#                 # cfg = [64],
#                 # cfg = [64,[64,64],64], # 끝에 linear classifier 하나 자동으로 붙습니다
#                 cfg = [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
#                 pre_trained_path = "net_save/save_now_net.pth",
#                 learning_rate = 0.00001,
#                 epoch_num = 200,
#                 verbose_interval = 10000, #숫자 크게 하면 꺼짐
#                 validation_interval = 50, #숫자 크게 하면 꺼짐
#                 tdBN_on = False,  # True # False
#                 BN_on = True,  # True # False
                
#                 surrogate = 'sigmoid', # 'rectangle' 'sigmoid' 'rough_rectangle'
                
#                 gradient_verbose = False,  # True # False  # weight gradient 각 layer마다 띄워줌

#                 BPTT_on = False,  # True # False

#                 scheduler_name = 'no', # 'no' 'StepLR' 'ExponentialLR' 'ReduceLROnPlateau' 'CosineAnnealingLR' 'OneCycleLR'
                
#                 ddp_on = False,
#                 )


# Files already downloaded and verified
# Files already downloaded and verified
# ==================================================
# Num of PARAMS: 9,302,410
# Memory: 35.49MiB at 32-bit
# ==================================================
# EPOCH 0
# train: 0-390/391 tr_acc: 27.50%, lr=['1e-05'], val_acc: 25.65%: 100%|██████████| 391/391 [08:39<00:00,  1.33s/it]
# epoch_time: 519.6511018276215 seconds


# EPOCH 1

# train: 1-390/391 tr_acc: 32.50%, lr=['1e-05'], val_acc: 32.00%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 520.4912984371185 seconds


# EPOCH 2

# train: 2-390/391 tr_acc: 33.75%, lr=['1e-05'], val_acc: 35.14%: 100%|██████████| 391/391 [08:33<00:00,  1.31s/it]
# epoch_time: 513.511162519455 seconds


# EPOCH 3

# train: 3-390/391 tr_acc: 28.75%, lr=['1e-05'], val_acc: 37.17%: 100%|██████████| 391/391 [08:30<00:00,  1.30s/it]
# epoch_time: 510.2808663845062 seconds


# EPOCH 4

# train: 4-390/391 tr_acc: 58.75%, lr=['1e-05'], val_acc: 40.37%: 100%|██████████| 391/391 [08:21<00:00,  1.28s/it]
# epoch_time: 501.48126435279846 seconds


# EPOCH 5

# train: 5-390/391 tr_acc: 37.50%, lr=['1e-05'], val_acc: 42.59%: 100%|██████████| 391/391 [08:18<00:00,  1.28s/it]
# epoch_time: 498.8543312549591 seconds


# EPOCH 6

# train: 6-390/391 tr_acc: 35.00%, lr=['1e-05'], val_acc: 43.45%: 100%|██████████| 391/391 [08:15<00:00,  1.27s/it]
# epoch_time: 495.6830530166626 seconds


# EPOCH 7

# train: 7-390/391 tr_acc: 48.75%, lr=['1e-05'], val_acc: 44.97%: 100%|██████████| 391/391 [08:16<00:00,  1.27s/it]
# epoch_time: 496.7004475593567 seconds


# EPOCH 8

# train: 8-390/391 tr_acc: 47.50%, lr=['1e-05'], val_acc: 46.19%: 100%|██████████| 391/391 [08:16<00:00,  1.27s/it]
# epoch_time: 496.552987575531 seconds


# EPOCH 9

# train: 9-390/391 tr_acc: 46.25%, lr=['1e-05'], val_acc: 46.52%: 100%|██████████| 391/391 [08:37<00:00,  1.32s/it]
# epoch_time: 517.5774216651917 seconds


# EPOCH 10

# train: 10-390/391 tr_acc: 48.75%, lr=['1e-05'], val_acc: 48.67%: 100%|██████████| 391/391 [08:57<00:00,  1.37s/it]
# epoch_time: 537.5675616264343 seconds


# EPOCH 11

# train: 11-390/391 tr_acc: 48.75%, lr=['1e-05'], val_acc: 48.40%: 100%|██████████| 391/391 [08:53<00:00,  1.37s/it]
# epoch_time: 533.9970047473907 seconds


# EPOCH 12

# train: 12-390/391 tr_acc: 43.75%, lr=['1e-05'], val_acc: 50.58%: 100%|██████████| 391/391 [08:57<00:00,  1.38s/it]
# epoch_time: 537.9225826263428 seconds


# EPOCH 13

# train: 13-390/391 tr_acc: 42.50%, lr=['1e-05'], val_acc: 50.98%: 100%|██████████| 391/391 [08:58<00:00,  1.38s/it]
# epoch_time: 538.5700080394745 seconds


# EPOCH 14

# train: 14-390/391 tr_acc: 48.75%, lr=['1e-05'], val_acc: 52.68%: 100%|██████████| 391/391 [08:59<00:00,  1.38s/it]
# epoch_time: 539.7910151481628 seconds


# EPOCH 15

# train: 15-390/391 tr_acc: 55.00%, lr=['1e-05'], val_acc: 54.05%: 100%|██████████| 391/391 [08:52<00:00,  1.36s/it]
# epoch_time: 532.8848164081573 seconds


# EPOCH 16

# train: 16-390/391 tr_acc: 52.50%, lr=['1e-05'], val_acc: 53.80%: 100%|██████████| 391/391 [08:58<00:00,  1.38s/it]
# epoch_time: 538.2881193161011 seconds


# EPOCH 17

# train: 17-390/391 tr_acc: 57.50%, lr=['1e-05'], val_acc: 54.73%: 100%|██████████| 391/391 [09:00<00:00,  1.38s/it]
# epoch_time: 540.2721989154816 seconds


# EPOCH 18

# train: 18-390/391 tr_acc: 60.00%, lr=['1e-05'], val_acc: 55.54%: 100%|██████████| 391/391 [09:00<00:00,  1.38s/it]
# epoch_time: 540.7067131996155 seconds


# EPOCH 19

# train: 19-390/391 tr_acc: 61.25%, lr=['1e-05'], val_acc: 56.20%: 100%|██████████| 391/391 [08:53<00:00,  1.36s/it]
# epoch_time: 533.882045507431 seconds


# EPOCH 20

# train: 20-390/391 tr_acc: 61.25%, lr=['1e-05'], val_acc: 57.64%: 100%|██████████| 391/391 [08:55<00:00,  1.37s/it]
# epoch_time: 535.4623730182648 seconds


# EPOCH 21

# train: 21-390/391 tr_acc: 57.50%, lr=['1e-05'], val_acc: 57.87%: 100%|██████████| 391/391 [08:57<00:00,  1.37s/it]
# epoch_time: 537.7847683429718 seconds


# EPOCH 22

# train: 22-390/391 tr_acc: 57.50%, lr=['1e-05'], val_acc: 58.46%: 100%|██████████| 391/391 [08:55<00:00,  1.37s/it]
# epoch_time: 535.7902438640594 seconds


# EPOCH 23

# train: 23-390/391 tr_acc: 57.50%, lr=['1e-05'], val_acc: 57.95%: 100%|██████████| 391/391 [08:57<00:00,  1.37s/it]
# epoch_time: 537.5092451572418 seconds


# EPOCH 24

# train: 24-390/391 tr_acc: 55.00%, lr=['1e-05'], val_acc: 57.98%: 100%|██████████| 391/391 [08:44<00:00,  1.34s/it]
# epoch_time: 524.3707118034363 seconds


# EPOCH 25

# train: 25-390/391 tr_acc: 50.00%, lr=['1e-05'], val_acc: 60.33%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.3296625614166 seconds


# EPOCH 26

# train: 26-390/391 tr_acc: 52.50%, lr=['1e-05'], val_acc: 60.02%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.8723931312561 seconds


# EPOCH 27

# train: 27-390/391 tr_acc: 60.00%, lr=['1e-05'], val_acc: 60.80%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.6468982696533 seconds


# EPOCH 28

# train: 28-390/391 tr_acc: 56.25%, lr=['1e-05'], val_acc: 60.99%: 100%|██████████| 391/391 [08:43<00:00,  1.34s/it]
# epoch_time: 524.2045466899872 seconds


# EPOCH 29

# train: 29-390/391 tr_acc: 66.25%, lr=['1e-05'], val_acc: 62.56%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 523.102609872818 seconds


# EPOCH 30

# train: 30-390/391 tr_acc: 65.00%, lr=['1e-05'], val_acc: 62.23%: 100%|██████████| 391/391 [08:43<00:00,  1.34s/it]
# epoch_time: 523.3475232124329 seconds


# EPOCH 31

# train: 31-390/391 tr_acc: 70.00%, lr=['1e-05'], val_acc: 61.05%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.611293554306 seconds


# EPOCH 32

# train: 32-390/391 tr_acc: 57.50%, lr=['1e-05'], val_acc: 63.73%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.550628900528 seconds


# EPOCH 33

# train: 33-390/391 tr_acc: 67.50%, lr=['1e-05'], val_acc: 63.01%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 520.7382657527924 seconds


# EPOCH 34

# train: 34-390/391 tr_acc: 66.25%, lr=['1e-05'], val_acc: 63.66%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 523.2350842952728 seconds


# EPOCH 35

# train: 35-390/391 tr_acc: 65.00%, lr=['1e-05'], val_acc: 64.20%: 100%|██████████| 391/391 [08:39<00:00,  1.33s/it]
# epoch_time: 519.9772260189056 seconds


# EPOCH 36

# train: 36-390/391 tr_acc: 61.25%, lr=['1e-05'], val_acc: 64.67%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 522.0173165798187 seconds


# EPOCH 37

# train: 37-390/391 tr_acc: 62.50%, lr=['1e-05'], val_acc: 64.44%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 522.151460647583 seconds


# EPOCH 38

# train: 38-390/391 tr_acc: 62.50%, lr=['1e-05'], val_acc: 64.77%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.8787703514099 seconds


# EPOCH 39

# train: 39-390/391 tr_acc: 66.25%, lr=['1e-05'], val_acc: 65.22%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.234944820404 seconds


# EPOCH 40

# train: 40-390/391 tr_acc: 68.75%, lr=['1e-05'], val_acc: 64.49%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.9287745952606 seconds


# EPOCH 41

# train: 41-390/391 tr_acc: 66.25%, lr=['1e-05'], val_acc: 66.06%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.8189256191254 seconds


# EPOCH 42

# train: 42-390/391 tr_acc: 76.25%, lr=['1e-05'], val_acc: 65.47%: 100%|██████████| 391/391 [08:43<00:00,  1.34s/it]
# epoch_time: 523.2214741706848 seconds


# EPOCH 43

# train: 43-390/391 tr_acc: 78.75%, lr=['1e-05'], val_acc: 66.97%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.9151771068573 seconds


# EPOCH 44

# train: 44-390/391 tr_acc: 72.50%, lr=['1e-05'], val_acc: 66.76%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 520.2902402877808 seconds


# EPOCH 45

# train: 45-390/391 tr_acc: 67.50%, lr=['1e-05'], val_acc: 66.30%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 520.3684139251709 seconds


# EPOCH 46

# train: 46-390/391 tr_acc: 76.25%, lr=['1e-05'], val_acc: 66.27%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.4130618572235 seconds


# EPOCH 47

# train: 47-390/391 tr_acc: 67.50%, lr=['1e-05'], val_acc: 66.20%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.4402587413788 seconds


# EPOCH 48

# train: 48-390/391 tr_acc: 67.50%, lr=['1e-05'], val_acc: 67.45%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 521.1478617191315 seconds


# EPOCH 49

# train: 49-390/391 tr_acc: 77.50%, lr=['1e-05'], val_acc: 68.14%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 520.3577303886414 seconds


# EPOCH 50

# train: 50-390/391 tr_acc: 63.75%, lr=['1e-05'], val_acc: 67.95%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.6194558143616 seconds


# EPOCH 51

# train: 51-390/391 tr_acc: 75.00%, lr=['1e-05'], val_acc: 68.41%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 520.8790509700775 seconds


# EPOCH 52

# train: 52-390/391 tr_acc: 66.25%, lr=['1e-05'], val_acc: 67.84%: 100%|██████████| 391/391 [08:40<00:00,  1.33s/it]
# epoch_time: 521.1223595142365 seconds


# EPOCH 53

# train: 53-390/391 tr_acc: 68.75%, lr=['1e-05'], val_acc: 68.00%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.3871352672577 seconds


# EPOCH 54

# train: 54-390/391 tr_acc: 73.75%, lr=['1e-05'], val_acc: 68.75%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.407853603363 seconds


# EPOCH 55

# train: 55-390/391 tr_acc: 71.25%, lr=['1e-05'], val_acc: 69.21%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.9369058609009 seconds


# EPOCH 56

# train: 56-390/391 tr_acc: 67.50%, lr=['1e-05'], val_acc: 69.28%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.430141210556 seconds


# EPOCH 57

# train: 57-390/391 tr_acc: 65.00%, lr=['1e-05'], val_acc: 69.16%: 100%|██████████| 391/391 [08:43<00:00,  1.34s/it]
# epoch_time: 523.3304183483124 seconds


# EPOCH 58

# train: 58-390/391 tr_acc: 67.50%, lr=['1e-05'], val_acc: 68.30%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.6461317539215 seconds


# EPOCH 59

# train: 59-390/391 tr_acc: 70.00%, lr=['1e-05'], val_acc: 69.50%: 100%|██████████| 391/391 [08:39<00:00,  1.33s/it]
# epoch_time: 520.0395126342773 seconds


# EPOCH 60

# train: 60-390/391 tr_acc: 68.75%, lr=['1e-05'], val_acc: 69.68%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.3599922657013 seconds


# EPOCH 61

# train: 61-390/391 tr_acc: 63.75%, lr=['1e-05'], val_acc: 70.64%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.446202993393 seconds


# EPOCH 62

# train: 62-390/391 tr_acc: 75.00%, lr=['1e-05'], val_acc: 70.57%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.744499206543 seconds


# EPOCH 63

# train: 63-390/391 tr_acc: 65.00%, lr=['1e-05'], val_acc: 70.47%: 100%|██████████| 391/391 [08:39<00:00,  1.33s/it]
# epoch_time: 519.3143074512482 seconds


# EPOCH 64

# train: 64-390/391 tr_acc: 63.75%, lr=['1e-05'], val_acc: 70.40%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 521.6692788600922 seconds


# EPOCH 65

# train: 65-390/391 tr_acc: 80.00%, lr=['1e-05'], val_acc: 70.64%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.667236328125 seconds


# EPOCH 66

# train: 66-390/391 tr_acc: 72.50%, lr=['1e-05'], val_acc: 69.29%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 522.0747861862183 seconds


# EPOCH 67

# train: 67-390/391 tr_acc: 73.75%, lr=['1e-05'], val_acc: 70.66%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.6984555721283 seconds


# EPOCH 68

# train: 68-390/391 tr_acc: 71.25%, lr=['1e-05'], val_acc: 70.71%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 523.1850342750549 seconds


# EPOCH 69

# train: 69-390/391 tr_acc: 86.25%, lr=['1e-05'], val_acc: 70.95%: 100%|██████████| 391/391 [08:41<00:00,  1.33s/it]
# epoch_time: 522.0473079681396 seconds


# EPOCH 70

# train: 70-390/391 tr_acc: 70.00%, lr=['1e-05'], val_acc: 71.07%: 100%|██████████| 391/391 [08:44<00:00,  1.34s/it]
# epoch_time: 524.3250975608826 seconds


# EPOCH 71

# train: 71-390/391 tr_acc: 71.25%, lr=['1e-05'], val_acc: 71.03%: 100%|██████████| 391/391 [08:43<00:00,  1.34s/it]
# epoch_time: 524.1436457633972 seconds


# EPOCH 72

# train: 72-390/391 tr_acc: 72.50%, lr=['1e-05'], val_acc: 70.97%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 522.3500926494598 seconds


# EPOCH 73

# train: 73-390/391 tr_acc: 81.25%, lr=['1e-05'], val_acc: 69.71%: 100%|██████████| 391/391 [08:43<00:00,  1.34s/it]
# epoch_time: 523.6104423999786 seconds


# EPOCH 74

# train: 74-390/391 tr_acc: 72.50%, lr=['1e-05'], val_acc: 71.35%: 100%|██████████| 391/391 [08:42<00:00,  1.34s/it]
# epoch_time: 523.1159906387329 seconds


# EPOCH 75

# train: 75-390/391 tr_acc: 70.00%, lr=['1e-05'], val_acc: 72.26%: 100%|██████████| 391/391 [08:43<00:00,  1.34s/it]
# epoch_time: 523.9588918685913 seconds


# EPOCH 76

# train: 76-105/391 tr_acc: 75.78%, lr=['1e-05'], val_acc: 71.69%:  27%|██▋       | 106/391 [02:24<06:28,  1.36s/it]

In [ ]:
# ### my_snn control board ########################
# epoch 6 acc 36

# decay = 0.25 # 0.875 0.25 0.125 0.75

# my_snn_system(  devices = "0,1,2,3,4,5",
#                 my_seed = 42,
#                 TIME = 8, # dvscifar 10
#                 BATCH = 128, # batch norm 할거면 2이상으로 해야함
#                 IMAGE_SIZE = 32, # dvscifar 48 # MNIST 28 # CIFAR10 32
#                 which_data = 'CIFAR10',# 'CIFAR10' 'MNIST' 'FASHION_MNIST' 'DVS-CIFAR10'
#                 CLASS_NUM = 10,
#                 data_path = '/data2', # YOU NEED TO CHANGE THIS
#                 rate_coding = False, # True # False

#                 lif_layer_v_init = 0.0,
#                 lif_layer_v_decay = decay,
#                 lif_layer_v_threshold = 0.5,
#                 lif_layer_v_reset = 0.0, #현재 안씀. 걍 빼기 해버림
#                 lif_layer_sg_width = 1.0, # surrogate sigmoid 쓸 때는 의미없음

#                 # synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
#                 synapse_conv_kernel_size = 3,
#                 synapse_conv_stride = 1,
#                 synapse_conv_padding = 1,
#                 synapse_conv_trace_const1 = 1,
#                 synapse_conv_trace_const2 = decay, # lif_layer_v_decay

#                 # synapse_fc_out_features = CLASS_NUM,
#                 synapse_fc_trace_const1 = 1,
#                 synapse_fc_trace_const2 = decay, # lif_layer_v_decay

#                 pre_trained = False, # True # False
#                 convTrue_fcFalse = True, # True # False
#                 # cfg = [64],
#                 # cfg = [64,[64,64],64], # 끝에 linear classifier 하나 자동으로 붙습니다
#                 # cfg = [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
#                 cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512],
#                 # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512, 'L', 4096, 4096],
#                 net_print = False, # True # False
#                 pre_trained_path = "net_save/save_now_net.pth",
#                 learning_rate = 0.00001,
#                 epoch_num = 200,
#                 verbose_interval = 10, #숫자 크게 하면 꺼짐
#                 validation_interval = 50, #숫자 크게 하면 꺼짐
#                 tdBN_on = False,  # True # False
#                 BN_on = True,  # True # False
                
#                 surrogate = 'sigmoid', # 'rectangle' 'sigmoid' 'rough_rectangle'
                
#                 gradient_verbose = False,  # True # False  # weight gradient 각 layer마다 띄워줌

#                 BPTT_on = True,  # True # False # True이면 BPTT, False이면 OTTT

#                 scheduler_name = 'no', # 'no' 'StepLR' 'ExponentialLR' 'ReduceLROnPlateau' 'CosineAnnealingLR' 'OneCycleLR'
                
#                 ddp_on = False,
#                 )

In [ ]:


# ### my_snn control board ########################
# decay = 0.25 # 0.875 0.25 0.125 0.75

# my_snn_system(  devices = "0,1,2,3,4,5",
#                 my_seed = 42,
#                 TIME = 8, # dvscifar 10
#                 BATCH = 128, # batch norm 할거면 2이상으로 해야함
#                 IMAGE_SIZE = 32, # dvscifar 48 # MNIST 28 # CIFAR10 32
#                 which_data = 'CIFAR10',# 'CIFAR10' 'MNIST' 'FASHION_MNIST' 'DVS-CIFAR10'
#                 CLASS_NUM = 10,
#                 data_path = '/data2', # YOU NEED TO CHANGE THIS
#                 rate_coding = False, # True # False

#                 lif_layer_v_init = 0.0,
#                 lif_layer_v_decay = decay,
#                 lif_layer_v_threshold = 0.5,
#                 lif_layer_v_reset = 0.0, #현재 안씀. 걍 빼기 해버림
#                 lif_layer_sg_width = 1.0, # surrogate sigmoid 쓸 때는 의미없음

#                 # synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
#                 synapse_conv_kernel_size = 3,
#                 synapse_conv_stride = 1,
#                 synapse_conv_padding = 1,
#                 synapse_conv_trace_const1 = 1,
#                 synapse_conv_trace_const2 = decay, # lif_layer_v_decay

#                 # synapse_fc_out_features = CLASS_NUM,
#                 synapse_fc_trace_const1 = 1,
#                 synapse_fc_trace_const2 = decay, # lif_layer_v_decay

#                 pre_trained = False, # True # False
#                 convTrue_fcFalse = True, # True # False

#                 # 'P' for average pooling, 'M' for maxpooling, 'L' for linear classifier, [  ] for residual block
#                 # cfg = [64],
#                 # cfg = [64,[64,64],64], # 끝에 linear classifier 하나 자동으로 붙습니다
#                 # cfg = [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
#                 cfg = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512],
#                 # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512],
#                 # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512, 'L', 4096, 4096],


#                 net_print = False, # True # False
#                 pre_trained_path = "net_save/save_now_net.pth",
#                 learning_rate = 0.00001,
#                 epoch_num = 200,
#                 verbose_interval = 50, #숫자 크게 하면 꺼짐
#                 validation_interval = 50, #숫자 크게 하면 꺼짐
#                 tdBN_on = False,  # True # False
#                 BN_on = True,  # True # False
                
#                 surrogate = 'sigmoid', # 'rectangle' 'sigmoid' 'rough_rectangle'
                
#                 gradient_verbose = False,  # True # False  # weight gradient 각 layer마다 띄워줌

#                 BPTT_on = False,  # True # False # True이면 BPTT, False이면 OTTT

#                 scheduler_name = 'no', # 'no' 'StepLR' 'ExponentialLR' 'ReduceLROnPlateau' 'CosineAnnealingLR' 'OneCycleLR'
                
#                 ddp_on = False,
#                 )
# # sigmoid와 BN이 있어야 잘되는건가? average pooling도 중요?
# # nda에서는 decay = 0.25, threshold = 0.5, width =1, surrogate = rectangle
# ## OTTT 에서는 decay = 0.5, threshold = 1.0, surrogate = sigmoid


# EPOCH 0
# train: 0-8/391 tr_acc: 16.41%, lr=['1e-05'], val_acc: 0.00%:   2%|▏         | 9/391 [00:24<07:31,  1.18s/it]  
# 0-9 training acc: 8.59%, lr=['1e-05'], val_acc: 0.00%
# train: 0-9/391 tr_acc: 8.59%, lr=['1e-05'], val_acc: 0.00%:   3%|▎         | 10/391 [00:25<06:58,  1.10s/it]
# iter_one_train_time: 0.9021003246307373 seconds, last one_val_time: 0 seconds

# train: 0-18/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 0.00%:   5%|▍         | 19/391 [00:36<08:32,  1.38s/it]
# 0-19 training acc: 7.81%, lr=['1e-05'], val_acc: 0.00%
# train: 0-19/391 tr_acc: 7.81%, lr=['1e-05'], val_acc: 0.00%:   5%|▌         | 20/391 [00:37<08:56,  1.45s/it] 
# iter_one_train_time: 1.6002912521362305 seconds, last one_val_time: 0 seconds

# train: 0-28/391 tr_acc: 9.38%, lr=['1e-05'], val_acc: 0.00%:   7%|▋         | 29/391 [00:51<09:12,  1.53s/it] 
# 0-29 training acc: 11.72%, lr=['1e-05'], val_acc: 0.00%
# train: 0-29/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 0.00%:   8%|▊         | 30/391 [00:52<09:10,  1.53s/it]
# iter_one_train_time: 1.5201919078826904 seconds, last one_val_time: 0 seconds

# train: 0-38/391 tr_acc: 14.06%, lr=['1e-05'], val_acc: 0.00%:  10%|▉         | 39/391 [01:06<08:56,  1.52s/it]
# 0-39 training acc: 11.72%, lr=['1e-05'], val_acc: 0.00%
# train: 0-39/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 0.00%:  10%|█         | 40/391 [01:08<08:56,  1.53s/it]
# iter_one_train_time: 1.529097318649292 seconds, last one_val_time: 0 seconds

# train: 0-48/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 0.00%:  13%|█▎        | 49/391 [01:22<08:52,  1.56s/it]
# 0-49 training acc: 10.16%, lr=['1e-05'], val_acc: 0.00%
# iter_one_train_time: 1.5265469551086426 seconds, last one_val_time: 0 seconds

# train: 0-58/391 tr_acc: 14.06%, lr=['1e-05'], val_acc: 11.72%:  15%|█▌        | 59/391 [01:55<09:53,  1.79s/it]
# 0-59 training acc: 9.38%, lr=['1e-05'], val_acc: 11.72%
# train: 0-59/391 tr_acc: 9.38%, lr=['1e-05'], val_acc: 11.72%:  15%|█▌        | 60/391 [01:56<09:26,  1.71s/it] 
# iter_one_train_time: 1.5334508419036865 seconds, last one_val_time: 16.866817474365234 seconds

# train: 0-68/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 11.72%:  18%|█▊        | 69/391 [02:10<08:27,  1.57s/it]
# 0-69 training acc: 12.50%, lr=['1e-05'], val_acc: 11.72%
# train: 0-69/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 11.72%:  18%|█▊        | 70/391 [02:12<08:21,  1.56s/it]
# iter_one_train_time: 1.5315065383911133 seconds, last one_val_time: 16.866817474365234 seconds

# train: 0-78/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 11.72%:  20%|██        | 79/391 [02:26<08:02,  1.55s/it]
# 0-79 training acc: 12.50%, lr=['1e-05'], val_acc: 11.72%
# train: 0-79/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 11.72%:  20%|██        | 80/391 [02:27<07:59,  1.54s/it]
# iter_one_train_time: 1.5302252769470215 seconds, last one_val_time: 16.866817474365234 seconds

# train: 0-88/391 tr_acc: 10.94%, lr=['1e-05'], val_acc: 11.72%:  23%|██▎       | 89/391 [02:41<07:48,  1.55s/it]
# 0-89 training acc: 12.50%, lr=['1e-05'], val_acc: 11.72%
# train: 0-89/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 11.72%:  23%|██▎       | 90/391 [02:43<07:45,  1.55s/it]
# iter_one_train_time: 1.5380234718322754 seconds, last one_val_time: 16.866817474365234 seconds

# train: 0-98/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 11.72%:  25%|██▌       | 99/391 [02:57<07:42,  1.58s/it]
# 0-99 training acc: 9.38%, lr=['1e-05'], val_acc: 11.72%
# iter_one_train_time: 1.5485491752624512 seconds, last one_val_time: 16.866817474365234 seconds

# train: 0-108/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 13.49%:  28%|██▊       | 109/391 [03:29<07:57,  1.69s/it]
# 0-109 training acc: 10.16%, lr=['1e-05'], val_acc: 13.49%
# train: 0-109/391 tr_acc: 10.16%, lr=['1e-05'], val_acc: 13.49%:  28%|██▊       | 110/391 [03:30<07:20,  1.57s/it]
# iter_one_train_time: 1.2722880840301514 seconds, last one_val_time: 17.4084632396698 seconds

# train: 0-118/391 tr_acc: 8.59%, lr=['1e-05'], val_acc: 13.49%:  30%|███       | 119/391 [03:42<06:55,  1.53s/it] 
# 0-119 training acc: 14.84%, lr=['1e-05'], val_acc: 13.49%
# train: 0-119/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 13.49%:  31%|███       | 120/391 [03:44<06:57,  1.54s/it]
# iter_one_train_time: 1.5578389167785645 seconds, last one_val_time: 17.4084632396698 seconds

# train: 0-128/391 tr_acc: 10.94%, lr=['1e-05'], val_acc: 13.49%:  33%|███▎      | 129/391 [03:58<06:53,  1.58s/it]
# 0-129 training acc: 10.94%, lr=['1e-05'], val_acc: 13.49%
# train: 0-129/391 tr_acc: 10.94%, lr=['1e-05'], val_acc: 13.49%:  33%|███▎      | 130/391 [04:00<06:50,  1.57s/it]
# iter_one_train_time: 1.5546276569366455 seconds, last one_val_time: 17.4084632396698 seconds

# train: 0-138/391 tr_acc: 10.94%, lr=['1e-05'], val_acc: 13.49%:  36%|███▌      | 139/391 [04:14<06:38,  1.58s/it]
# 0-139 training acc: 10.94%, lr=['1e-05'], val_acc: 13.49%
# train: 0-139/391 tr_acc: 10.94%, lr=['1e-05'], val_acc: 13.49%:  36%|███▌      | 140/391 [04:15<06:34,  1.57s/it]
# iter_one_train_time: 1.5492908954620361 seconds, last one_val_time: 17.4084632396698 seconds

# train: 0-148/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 13.49%:  38%|███▊      | 149/391 [04:29<06:17,  1.56s/it]
# 0-149 training acc: 15.62%, lr=['1e-05'], val_acc: 13.49%
# iter_one_train_time: 1.5452179908752441 seconds, last one_val_time: 17.4084632396698 seconds

# train: 0-158/391 tr_acc: 10.16%, lr=['1e-05'], val_acc: 14.48%:  41%|████      | 159/391 [05:02<06:51,  1.77s/it]
# 0-159 training acc: 17.97%, lr=['1e-05'], val_acc: 14.48%
# train: 0-159/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 14.48%:  41%|████      | 160/391 [05:04<06:40,  1.74s/it]
# iter_one_train_time: 1.6436526775360107 seconds, last one_val_time: 17.05498194694519 seconds

# train: 0-168/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 14.48%:  43%|████▎     | 169/391 [05:18<05:54,  1.60s/it]
# 0-169 training acc: 16.41%, lr=['1e-05'], val_acc: 14.48%
# train: 0-169/391 tr_acc: 16.41%, lr=['1e-05'], val_acc: 14.48%:  43%|████▎     | 170/391 [05:20<05:50,  1.59s/it]
# iter_one_train_time: 1.5567233562469482 seconds, last one_val_time: 17.05498194694519 seconds

# train: 0-178/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 14.48%:  46%|████▌     | 179/391 [05:34<05:30,  1.56s/it]
# 0-179 training acc: 18.75%, lr=['1e-05'], val_acc: 14.48%
# train: 0-179/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 14.48%:  46%|████▌     | 180/391 [05:35<05:28,  1.56s/it]
# iter_one_train_time: 1.546079158782959 seconds, last one_val_time: 17.05498194694519 seconds

# train: 0-188/391 tr_acc: 9.38%, lr=['1e-05'], val_acc: 14.48%:  48%|████▊     | 189/391 [05:49<05:15,  1.56s/it] 
# 0-189 training acc: 13.28%, lr=['1e-05'], val_acc: 14.48%
# train: 0-189/391 tr_acc: 13.28%, lr=['1e-05'], val_acc: 14.48%:  49%|████▊     | 190/391 [05:51<05:12,  1.56s/it]
# iter_one_train_time: 1.5421521663665771 seconds, last one_val_time: 17.05498194694519 seconds

# train: 0-198/391 tr_acc: 22.66%, lr=['1e-05'], val_acc: 14.48%:  51%|█████     | 199/391 [06:05<04:58,  1.56s/it]
# 0-199 training acc: 14.84%, lr=['1e-05'], val_acc: 14.48%
# iter_one_train_time: 1.6056420803070068 seconds, last one_val_time: 17.05498194694519 seconds

# train: 0-208/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 15.04%:  53%|█████▎    | 209/391 [06:39<05:28,  1.80s/it]
# 0-209 training acc: 20.31%, lr=['1e-05'], val_acc: 15.04%
# train: 0-209/391 tr_acc: 20.31%, lr=['1e-05'], val_acc: 15.04%:  54%|█████▎    | 210/391 [06:40<05:13,  1.73s/it]
# iter_one_train_time: 1.5574557781219482 seconds, last one_val_time: 17.079707384109497 seconds

# train: 0-218/391 tr_acc: 14.06%, lr=['1e-05'], val_acc: 15.04%:  56%|█████▌    | 219/391 [06:54<04:29,  1.57s/it]
# 0-219 training acc: 17.97%, lr=['1e-05'], val_acc: 15.04%
# train: 0-219/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 15.04%:  56%|█████▋    | 220/391 [06:56<04:26,  1.56s/it]
# iter_one_train_time: 1.5369503498077393 seconds, last one_val_time: 17.079707384109497 seconds

# train: 0-228/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 15.04%:  59%|█████▊    | 229/391 [07:10<04:16,  1.58s/it]
# 0-229 training acc: 10.16%, lr=['1e-05'], val_acc: 15.04%
# train: 0-229/391 tr_acc: 10.16%, lr=['1e-05'], val_acc: 15.04%:  59%|█████▉    | 230/391 [07:12<04:12,  1.57s/it]
# iter_one_train_time: 1.5427579879760742 seconds, last one_val_time: 17.079707384109497 seconds

# train: 0-238/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 15.04%:  61%|██████    | 239/391 [07:26<03:56,  1.55s/it]
# 0-239 training acc: 20.31%, lr=['1e-05'], val_acc: 15.04%
# train: 0-239/391 tr_acc: 20.31%, lr=['1e-05'], val_acc: 15.04%:  61%|██████▏   | 240/391 [07:27<03:54,  1.55s/it]
# iter_one_train_time: 1.5445988178253174 seconds, last one_val_time: 17.079707384109497 seconds

# train: 0-248/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 15.04%:  64%|██████▎   | 249/391 [07:41<03:41,  1.56s/it]
# 0-249 training acc: 10.94%, lr=['1e-05'], val_acc: 15.04%
# iter_one_train_time: 1.5501785278320312 seconds, last one_val_time: 17.079707384109497 seconds

# train: 0-258/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 15.79%:  66%|██████▌   | 259/391 [08:14<03:54,  1.78s/it]
# 0-259 training acc: 17.97%, lr=['1e-05'], val_acc: 15.79%
# train: 0-259/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 15.79%:  66%|██████▋   | 260/391 [08:16<03:44,  1.71s/it]
# iter_one_train_time: 1.5513510704040527 seconds, last one_val_time: 16.802847623825073 seconds

# train: 0-268/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 15.79%:  69%|██████▉   | 269/391 [08:30<03:13,  1.59s/it]
# 0-269 training acc: 10.94%, lr=['1e-05'], val_acc: 15.79%
# train: 0-269/391 tr_acc: 10.94%, lr=['1e-05'], val_acc: 15.79%:  69%|██████▉   | 270/391 [08:32<03:10,  1.58s/it]
# iter_one_train_time: 1.545609712600708 seconds, last one_val_time: 16.802847623825073 seconds

# train: 0-278/391 tr_acc: 16.41%, lr=['1e-05'], val_acc: 15.79%:  71%|███████▏  | 279/391 [08:46<02:56,  1.58s/it]
# 0-279 training acc: 13.28%, lr=['1e-05'], val_acc: 15.79%
# train: 0-279/391 tr_acc: 13.28%, lr=['1e-05'], val_acc: 15.79%:  72%|███████▏  | 280/391 [08:47<02:54,  1.58s/it]
# iter_one_train_time: 1.5714669227600098 seconds, last one_val_time: 16.802847623825073 seconds

# train: 0-288/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 15.79%:  74%|███████▍  | 289/391 [09:01<02:38,  1.55s/it]
# 0-289 training acc: 21.88%, lr=['1e-05'], val_acc: 15.79%
# train: 0-289/391 tr_acc: 21.88%, lr=['1e-05'], val_acc: 15.79%:  74%|███████▍  | 290/391 [09:03<02:37,  1.56s/it]
# iter_one_train_time: 1.5540351867675781 seconds, last one_val_time: 16.802847623825073 seconds

# train: 0-298/391 tr_acc: 13.28%, lr=['1e-05'], val_acc: 15.79%:  76%|███████▋  | 299/391 [09:14<01:36,  1.05s/it]
# 0-299 training acc: 19.53%, lr=['1e-05'], val_acc: 15.79%
# iter_one_train_time: 0.8877673149108887 seconds, last one_val_time: 16.802847623825073 seconds

# train: 0-308/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 16.33%:  79%|███████▉  | 309/391 [09:40<01:30,  1.10s/it]
# 0-309 training acc: 18.75%, lr=['1e-05'], val_acc: 16.33%
# train: 0-309/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 16.33%:  79%|███████▉  | 310/391 [09:41<01:23,  1.04s/it]
# iter_one_train_time: 0.8837041854858398 seconds, last one_val_time: 16.87713122367859 seconds

# train: 0-318/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 16.33%:  82%|████████▏ | 319/391 [09:49<01:06,  1.09it/s]
# 0-319 training acc: 14.06%, lr=['1e-05'], val_acc: 16.33%
# train: 0-319/391 tr_acc: 14.06%, lr=['1e-05'], val_acc: 16.33%:  82%|████████▏ | 320/391 [09:50<01:05,  1.09it/s]
# iter_one_train_time: 0.9021422863006592 seconds, last one_val_time: 16.87713122367859 seconds

# train: 0-328/391 tr_acc: 21.88%, lr=['1e-05'], val_acc: 16.33%:  84%|████████▍ | 329/391 [09:58<00:55,  1.12it/s]
# 0-329 training acc: 12.50%, lr=['1e-05'], val_acc: 16.33%
# train: 0-329/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 16.33%:  84%|████████▍ | 330/391 [09:59<00:54,  1.11it/s]
# iter_one_train_time: 0.8932549953460693 seconds, last one_val_time: 16.87713122367859 seconds

# train: 0-338/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 16.33%:  87%|████████▋ | 339/391 [10:07<00:46,  1.12it/s]
# 0-339 training acc: 17.97%, lr=['1e-05'], val_acc: 16.33%
# train: 0-339/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 16.33%:  87%|████████▋ | 340/391 [10:08<00:45,  1.11it/s]
# iter_one_train_time: 0.9081242084503174 seconds, last one_val_time: 16.87713122367859 seconds

# train: 0-348/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 16.33%:  89%|████████▉ | 349/391 [10:17<00:38,  1.08it/s]
# 0-349 training acc: 17.19%, lr=['1e-05'], val_acc: 16.33%
# iter_one_train_time: 0.8888015747070312 seconds, last one_val_time: 16.87713122367859 seconds

# train: 0-358/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 17.23%:  92%|█████████▏| 359/391 [10:43<00:35,  1.11s/it]
# 0-359 training acc: 17.19%, lr=['1e-05'], val_acc: 17.23%
# train: 0-359/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 17.23%:  92%|█████████▏| 360/391 [10:44<00:32,  1.04s/it]
# iter_one_train_time: 0.889817476272583 seconds, last one_val_time: 17.09144353866577 seconds

# train: 0-368/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 17.23%:  94%|█████████▍| 369/391 [10:52<00:19,  1.12it/s]
# 0-369 training acc: 10.16%, lr=['1e-05'], val_acc: 17.23%
# train: 0-369/391 tr_acc: 10.16%, lr=['1e-05'], val_acc: 17.23%:  95%|█████████▍| 370/391 [10:53<00:19,  1.09it/s]
# iter_one_train_time: 0.9528336524963379 seconds, last one_val_time: 17.09144353866577 seconds

# train: 0-378/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 17.23%:  97%|█████████▋| 379/391 [11:01<00:10,  1.11it/s]
# 0-379 training acc: 16.41%, lr=['1e-05'], val_acc: 17.23%
# train: 0-379/391 tr_acc: 16.41%, lr=['1e-05'], val_acc: 17.23%:  97%|█████████▋| 380/391 [11:02<00:09,  1.11it/s]
# iter_one_train_time: 0.8962624073028564 seconds, last one_val_time: 17.09144353866577 seconds

# train: 0-388/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 17.23%:  99%|█████████▉| 389/391 [11:10<00:01,  1.10it/s]
# 0-389 training acc: 16.41%, lr=['1e-05'], val_acc: 17.23%
# train: 0-389/391 tr_acc: 16.41%, lr=['1e-05'], val_acc: 17.23%: 100%|█████████▉| 390/391 [11:11<00:00,  1.11it/s]
# iter_one_train_time: 0.8882513046264648 seconds, last one_val_time: 17.09144353866577 seconds

# train: 0-390/391 tr_acc: 16.25%, lr=['1e-05'], val_acc: 17.23%: 100%|██████████| 391/391 [11:12<00:00,  1.72s/it]
# epoch_time: 672.5876622200012 seconds


# EPOCH 1

# train: 1-8/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 17.23%:   2%|▏         | 9/391 [00:08<05:46,  1.10it/s]
# 1-9 training acc: 11.72%, lr=['1e-05'], val_acc: 17.23%
# train: 1-9/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 17.23%:   3%|▎         | 10/391 [00:09<05:45,  1.10it/s]
# iter_one_train_time: 0.9011766910552979 seconds, last one_val_time: 17.09144353866577 seconds

# train: 1-18/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 17.23%:   5%|▍         | 19/391 [00:17<05:45,  1.08it/s]
# 1-19 training acc: 14.06%, lr=['1e-05'], val_acc: 17.23%
# train: 1-19/391 tr_acc: 14.06%, lr=['1e-05'], val_acc: 17.23%:   5%|▌         | 20/391 [00:18<05:40,  1.09it/s]
# iter_one_train_time: 0.8899838924407959 seconds, last one_val_time: 17.09144353866577 seconds

# train: 1-28/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 17.23%:   7%|▋         | 29/391 [00:26<05:33,  1.09it/s]
# 1-29 training acc: 13.28%, lr=['1e-05'], val_acc: 17.23%
# train: 1-29/391 tr_acc: 13.28%, lr=['1e-05'], val_acc: 17.23%:   8%|▊         | 30/391 [00:27<05:32,  1.09it/s]
# iter_one_train_time: 0.913830041885376 seconds, last one_val_time: 17.09144353866577 seconds

# train: 1-38/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 17.23%:  10%|▉         | 39/391 [00:35<05:17,  1.11it/s]
# 1-39 training acc: 12.50%, lr=['1e-05'], val_acc: 17.23%
# train: 1-39/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 17.23%:  10%|█         | 40/391 [00:36<05:16,  1.11it/s]
# iter_one_train_time: 0.9035923480987549 seconds, last one_val_time: 17.09144353866577 seconds

# train: 1-48/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 17.23%:  13%|█▎        | 49/391 [00:44<05:15,  1.08it/s]
# 1-49 training acc: 14.06%, lr=['1e-05'], val_acc: 17.23%
# iter_one_train_time: 0.8847906589508057 seconds, last one_val_time: 17.09144353866577 seconds

# train: 1-58/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 17.51%:  15%|█▌        | 59/391 [01:10<06:04,  1.10s/it]
# 1-59 training acc: 15.62%, lr=['1e-05'], val_acc: 17.51%
# train: 1-59/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 17.51%:  15%|█▌        | 60/391 [01:11<05:43,  1.04s/it]
# iter_one_train_time: 0.8940188884735107 seconds, last one_val_time: 16.755418300628662 seconds

# train: 1-68/391 tr_acc: 25.00%, lr=['1e-05'], val_acc: 17.51%:  18%|█▊        | 69/391 [01:20<04:57,  1.08it/s]
# 1-69 training acc: 17.97%, lr=['1e-05'], val_acc: 17.51%
# train: 1-69/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 17.51%:  18%|█▊        | 70/391 [01:20<04:53,  1.09it/s]
# iter_one_train_time: 0.8915848731994629 seconds, last one_val_time: 16.755418300628662 seconds

# train: 1-78/391 tr_acc: 24.22%, lr=['1e-05'], val_acc: 17.51%:  20%|██        | 79/391 [01:29<04:44,  1.10it/s]
# 1-79 training acc: 18.75%, lr=['1e-05'], val_acc: 17.51%
# train: 1-79/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 17.51%:  20%|██        | 80/391 [01:30<04:40,  1.11it/s]
# iter_one_train_time: 0.8839564323425293 seconds, last one_val_time: 16.755418300628662 seconds

# train: 1-88/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 17.51%:  23%|██▎       | 89/391 [01:38<04:33,  1.10it/s]
# 1-89 training acc: 11.72%, lr=['1e-05'], val_acc: 17.51%
# train: 1-89/391 tr_acc: 11.72%, lr=['1e-05'], val_acc: 17.51%:  23%|██▎       | 90/391 [01:39<04:33,  1.10it/s]
# iter_one_train_time: 0.9085617065429688 seconds, last one_val_time: 16.755418300628662 seconds

# train: 1-98/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 17.51%:  25%|██▌       | 99/391 [01:47<04:50,  1.00it/s]
# 1-99 training acc: 17.19%, lr=['1e-05'], val_acc: 17.51%
# iter_one_train_time: 1.5453689098358154 seconds, last one_val_time: 16.755418300628662 seconds

# train: 1-108/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 18.53%:  28%|██▊       | 109/391 [02:21<08:22,  1.78s/it]
# 1-109 training acc: 22.66%, lr=['1e-05'], val_acc: 18.53%
# train: 1-109/391 tr_acc: 22.66%, lr=['1e-05'], val_acc: 18.53%:  28%|██▊       | 110/391 [02:22<08:01,  1.71s/it]
# iter_one_train_time: 1.5485820770263672 seconds, last one_val_time: 16.960761070251465 seconds

# train: 1-118/391 tr_acc: 10.16%, lr=['1e-05'], val_acc: 18.53%:  30%|███       | 119/391 [02:36<07:08,  1.57s/it]
# 1-119 training acc: 21.09%, lr=['1e-05'], val_acc: 18.53%
# train: 1-119/391 tr_acc: 21.09%, lr=['1e-05'], val_acc: 18.53%:  31%|███       | 120/391 [02:38<07:12,  1.60s/it]
# iter_one_train_time: 1.6461992263793945 seconds, last one_val_time: 16.960761070251465 seconds

# train: 1-128/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 18.53%:  33%|███▎      | 129/391 [02:52<06:57,  1.59s/it]
# 1-129 training acc: 17.97%, lr=['1e-05'], val_acc: 18.53%
# train: 1-129/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 18.53%:  33%|███▎      | 130/391 [02:54<06:55,  1.59s/it]
# iter_one_train_time: 1.5823149681091309 seconds, last one_val_time: 16.960761070251465 seconds

# train: 1-138/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 18.53%:  36%|███▌      | 139/391 [03:08<06:37,  1.58s/it]
# 1-139 training acc: 21.88%, lr=['1e-05'], val_acc: 18.53%
# train: 1-139/391 tr_acc: 21.88%, lr=['1e-05'], val_acc: 18.53%:  36%|███▌      | 140/391 [03:10<06:34,  1.57s/it]
# iter_one_train_time: 1.558037281036377 seconds, last one_val_time: 16.960761070251465 seconds

# train: 1-148/391 tr_acc: 18.75%, lr=['1e-05'], val_acc: 18.53%:  38%|███▊      | 149/391 [03:24<06:20,  1.57s/it]
# 1-149 training acc: 23.44%, lr=['1e-05'], val_acc: 18.53%
# iter_one_train_time: 1.5661795139312744 seconds, last one_val_time: 16.960761070251465 seconds

# train: 1-158/391 tr_acc: 22.66%, lr=['1e-05'], val_acc: 18.44%:  41%|████      | 159/391 [03:57<06:58,  1.80s/it]
# 1-159 training acc: 23.44%, lr=['1e-05'], val_acc: 18.44%
# train: 1-159/391 tr_acc: 23.44%, lr=['1e-05'], val_acc: 18.44%:  41%|████      | 160/391 [03:58<06:38,  1.72s/it]
# iter_one_train_time: 1.539853811264038 seconds, last one_val_time: 16.89861512184143 seconds

# train: 1-168/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 18.44%:  43%|████▎     | 169/391 [04:12<05:52,  1.59s/it]
# 1-169 training acc: 23.44%, lr=['1e-05'], val_acc: 18.44%
# train: 1-169/391 tr_acc: 23.44%, lr=['1e-05'], val_acc: 18.44%:  43%|████▎     | 170/391 [04:14<05:49,  1.58s/it]
# iter_one_train_time: 1.5540361404418945 seconds, last one_val_time: 16.89861512184143 seconds

# train: 1-178/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 18.44%:  46%|████▌     | 179/391 [04:28<05:34,  1.58s/it]
# 1-179 training acc: 24.22%, lr=['1e-05'], val_acc: 18.44%
# train: 1-179/391 tr_acc: 24.22%, lr=['1e-05'], val_acc: 18.44%:  46%|████▌     | 180/391 [04:30<05:33,  1.58s/it]
# iter_one_train_time: 1.5785167217254639 seconds, last one_val_time: 16.89861512184143 seconds

# train: 1-188/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 18.44%:  48%|████▊     | 189/391 [04:44<05:18,  1.58s/it]
# 1-189 training acc: 21.88%, lr=['1e-05'], val_acc: 18.44%
# train: 1-189/391 tr_acc: 21.88%, lr=['1e-05'], val_acc: 18.44%:  49%|████▊     | 190/391 [04:45<05:17,  1.58s/it]
# iter_one_train_time: 1.5851466655731201 seconds, last one_val_time: 16.89861512184143 seconds

# train: 1-198/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 18.44%:  51%|█████     | 199/391 [05:00<05:06,  1.60s/it]
# 1-199 training acc: 21.88%, lr=['1e-05'], val_acc: 18.44%
# iter_one_train_time: 1.5574944019317627 seconds, last one_val_time: 16.89861512184143 seconds

# train: 1-208/391 tr_acc: 19.53%, lr=['1e-05'], val_acc: 18.39%:  53%|█████▎    | 209/391 [05:26<03:25,  1.13s/it]
# 1-209 training acc: 15.62%, lr=['1e-05'], val_acc: 18.39%
# train: 1-209/391 tr_acc: 15.62%, lr=['1e-05'], val_acc: 18.39%:  54%|█████▎    | 210/391 [05:27<03:11,  1.06s/it]
# iter_one_train_time: 0.878171443939209 seconds, last one_val_time: 16.62509536743164 seconds

# train: 1-218/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 18.39%:  56%|█████▌    | 219/391 [05:35<02:35,  1.11it/s]
# 1-219 training acc: 19.53%, lr=['1e-05'], val_acc: 18.39%
# train: 1-219/391 tr_acc: 19.53%, lr=['1e-05'], val_acc: 18.39%:  56%|█████▋    | 220/391 [05:36<02:33,  1.12it/s]
# iter_one_train_time: 0.8779330253601074 seconds, last one_val_time: 16.62509536743164 seconds

# train: 1-228/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 18.39%:  59%|█████▊    | 229/391 [05:44<02:24,  1.12it/s]
# 1-229 training acc: 13.28%, lr=['1e-05'], val_acc: 18.39%
# train: 1-229/391 tr_acc: 13.28%, lr=['1e-05'], val_acc: 18.39%:  59%|█████▉    | 230/391 [05:45<02:22,  1.13it/s]
# iter_one_train_time: 0.8615732192993164 seconds, last one_val_time: 16.62509536743164 seconds

# train: 1-238/391 tr_acc: 21.88%, lr=['1e-05'], val_acc: 18.39%:  61%|██████    | 239/391 [05:53<02:17,  1.10it/s]
# 1-239 training acc: 24.22%, lr=['1e-05'], val_acc: 18.39%
# train: 1-239/391 tr_acc: 24.22%, lr=['1e-05'], val_acc: 18.39%:  61%|██████▏   | 240/391 [05:54<02:16,  1.11it/s]
# iter_one_train_time: 0.8894679546356201 seconds, last one_val_time: 16.62509536743164 seconds

# train: 1-248/391 tr_acc: 20.31%, lr=['1e-05'], val_acc: 18.39%:  64%|██████▎   | 249/391 [06:02<02:07,  1.11it/s]
# 1-249 training acc: 17.19%, lr=['1e-05'], val_acc: 18.39%
# iter_one_train_time: 0.9120488166809082 seconds, last one_val_time: 16.62509536743164 seconds

# train: 1-258/391 tr_acc: 20.31%, lr=['1e-05'], val_acc: 19.09%:  66%|██████▌   | 259/391 [06:28<02:26,  1.11s/it]
# 1-259 training acc: 16.41%, lr=['1e-05'], val_acc: 19.09%
# train: 1-259/391 tr_acc: 16.41%, lr=['1e-05'], val_acc: 19.09%:  66%|██████▋   | 260/391 [06:29<02:17,  1.05s/it]
# iter_one_train_time: 0.8989126682281494 seconds, last one_val_time: 16.488541841506958 seconds

# train: 1-268/391 tr_acc: 17.97%, lr=['1e-05'], val_acc: 19.09%:  69%|██████▉   | 269/391 [06:39<02:36,  1.28s/it]
# 1-269 training acc: 16.41%, lr=['1e-05'], val_acc: 19.09%
# train: 1-269/391 tr_acc: 16.41%, lr=['1e-05'], val_acc: 19.09%:  69%|██████▉   | 270/391 [06:40<02:44,  1.36s/it]
# iter_one_train_time: 1.5264859199523926 seconds, last one_val_time: 16.488541841506958 seconds

# train: 1-278/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 19.09%:  71%|███████▏  | 279/391 [06:54<02:54,  1.56s/it]
# 1-279 training acc: 21.09%, lr=['1e-05'], val_acc: 19.09%
# train: 1-279/391 tr_acc: 21.09%, lr=['1e-05'], val_acc: 19.09%:  72%|███████▏  | 280/391 [06:56<02:53,  1.56s/it]
# iter_one_train_time: 1.564835786819458 seconds, last one_val_time: 16.488541841506958 seconds

# train: 1-288/391 tr_acc: 14.84%, lr=['1e-05'], val_acc: 19.09%:  74%|███████▍  | 289/391 [07:10<02:39,  1.57s/it]
# 1-289 training acc: 23.44%, lr=['1e-05'], val_acc: 19.09%
# train: 1-289/391 tr_acc: 23.44%, lr=['1e-05'], val_acc: 19.09%:  74%|███████▍  | 290/391 [07:11<02:37,  1.56s/it]
# iter_one_train_time: 1.5514628887176514 seconds, last one_val_time: 16.488541841506958 seconds


In [ ]:
# ### my_snn control board ########################
# decay = 0.25 # 0.875 0.25 0.125 0.75

# my_snn_system(  devices = "0,1,2,3,4,5",
#                 my_seed = 42,
#                 TIME = 8, # dvscifar 10
#                 BATCH = 128, # batch norm 할거면 2이상으로 해야함
#                 IMAGE_SIZE = 32, # dvscifar 48 # MNIST 28 # CIFAR10 32
#                 which_data = 'CIFAR10',# 'CIFAR10' 'MNIST' 'FASHION_MNIST' 'DVS-CIFAR10'
#                 CLASS_NUM = 10,
#                 data_path = '/data2', # YOU NEED TO CHANGE THIS
#                 rate_coding = False, # True # False

#                 lif_layer_v_init = 0.0,
#                 lif_layer_v_decay = decay,
#                 lif_layer_v_threshold = 0.5,
#                 lif_layer_v_reset = 0.0, #현재 안씀. 걍 빼기 해버림
#                 lif_layer_sg_width = 1.0, # surrogate sigmoid 쓸 때는 의미없음

#                 # synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
#                 synapse_conv_kernel_size = 3,
#                 synapse_conv_stride = 1,
#                 synapse_conv_padding = 1,
#                 synapse_conv_trace_const1 = 1,
#                 synapse_conv_trace_const2 = decay, # lif_layer_v_decay

#                 # synapse_fc_out_features = CLASS_NUM,
#                 synapse_fc_trace_const1 = 1,
#                 synapse_fc_trace_const2 = decay, # lif_layer_v_decay

#                 pre_trained = False, # True # False
#                 convTrue_fcFalse = True, # True # False

#                 # 'P' for average pooling, 'M' for maxpooling, 'L' for linear classifier, [  ] for residual block
#                 # cfg = [64],
#                 # cfg = [64,[64,64],64], # 끝에 linear classifier 하나 자동으로 붙습니다
#                 cfg = [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
#                 # cfg = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512],
#                 # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512],
#                 # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512, 'L', 4096, 4096],


#                 net_print = False, # True # False
#                 pre_trained_path = "net_save/save_now_net.pth",
#                 learning_rate = 0.00001,
#                 epoch_num = 200,
#                 verbose_interval = 50, #숫자 크게 하면 꺼짐
#                 validation_interval = 50, #숫자 크게 하면 꺼짐
#                 tdBN_on = False,  # True # False
#                 BN_on = True,  # True # False
                
#                 surrogate = 'sigmoid', # 'rectangle' 'sigmoid' 'rough_rectangle'
                
#                 gradient_verbose = False,  # True # False  # weight gradient 각 layer마다 띄워줌

#                 BPTT_on = False,  # True # False # True이면 BPTT, False이면 OTTT

#                 scheduler_name = 'no', # 'no' 'StepLR' 'ExponentialLR' 'ReduceLROnPlateau' 'CosineAnnealingLR' 'OneCycleLR'
                
#                 ddp_on = False,
#                 )
# # sigmoid와 BN이 있어야 잘되는건가? average pooling도 중요?
# # nda에서는 decay = 0.25, threshold = 0.5, width =1, surrogate = rectangle
# ## OTTT 에서는 decay = 0.5, threshold = 1.0, surrogate = sigmoid



# Files already downloaded and verified
# Files already downloaded and verified
# ==================================================
# My Num of PARAMS: 9,302,410, system's param_num : 9,346,442
# Memory: 35.49MiB at 32-bit
# ==================================================
# EPOCH 0
# train: 0-48/391 tr_acc: 12.50%, lr=['1e-05'], val_acc: 0.00%:  13%|█▎        | 49/391 [01:31<05:53,  1.03s/it]
# 0-49 training acc: 10.16%, lr=['1e-05'], val_acc: 0.00%
# iter_one_train_time: 1.548201084136963 seconds, last one_val_time: 0 seconds

# train: 0-98/391 tr_acc: 19.53%, lr=['1e-05'], val_acc: 16.11%:  25%|██▌       | 99/391 [03:07<07:44,  1.59s/it]
# 0-99 training acc: 14.84%, lr=['1e-05'], val_acc: 16.11%
# iter_one_train_time: 1.5574898719787598 seconds, last one_val_time: 16.978580951690674 seconds

# train: 0-148/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 18.27%:  38%|███▊      | 149/391 [04:44<06:22,  1.58s/it]
# 0-149 training acc: 14.06%, lr=['1e-05'], val_acc: 18.27%
# iter_one_train_time: 1.5439555644989014 seconds, last one_val_time: 16.892942667007446 seconds

# train: 0-198/391 tr_acc: 26.56%, lr=['1e-05'], val_acc: 19.09%:  51%|█████     | 199/391 [05:53<02:52,  1.11it/s]
# 0-199 training acc: 25.00%, lr=['1e-05'], val_acc: 19.09%
# iter_one_train_time: 0.9714524745941162 seconds, last one_val_time: 16.94005250930786 seconds

# train: 0-248/391 tr_acc: 20.31%, lr=['1e-05'], val_acc: 21.53%:  64%|██████▎   | 249/391 [07:28<03:42,  1.57s/it]
# 0-249 training acc: 21.09%, lr=['1e-05'], val_acc: 21.53%
# iter_one_train_time: 1.5712730884552002 seconds, last one_val_time: 16.967337608337402 seconds

# train: 0-298/391 tr_acc: 21.88%, lr=['1e-05'], val_acc: 22.16%:  76%|███████▋  | 299/391 [09:03<02:23,  1.56s/it]
# 0-299 training acc: 20.31%, lr=['1e-05'], val_acc: 22.16%
# iter_one_train_time: 1.0139188766479492 seconds, last one_val_time: 16.793952465057373 seconds

# train: 0-348/391 tr_acc: 23.44%, lr=['1e-05'], val_acc: 22.73%:  89%|████████▉ | 349/391 [10:38<01:06,  1.57s/it]
# 0-349 training acc: 21.09%, lr=['1e-05'], val_acc: 22.73%
# iter_one_train_time: 1.538879156112671 seconds, last one_val_time: 17.2120463848114 seconds

# train: 0-390/391 tr_acc: 26.25%, lr=['1e-05'], val_acc: 23.78%: 100%|██████████| 391/391 [12:01<00:00,  1.84s/it]
# epoch_time: 721.2595112323761 seconds


# EPOCH 1

# train: 1-48/391 tr_acc: 22.66%, lr=['1e-05'], val_acc: 23.78%:  13%|█▎        | 49/391 [01:15<08:49,  1.55s/it]
# 1-49 training acc: 22.66%, lr=['1e-05'], val_acc: 23.78%
# iter_one_train_time: 1.547459363937378 seconds, last one_val_time: 17.239224672317505 seconds

# train: 1-98/391 tr_acc: 22.66%, lr=['1e-05'], val_acc: 23.63%:  25%|██▌       | 99/391 [02:47<04:46,  1.02it/s]
# 1-99 training acc: 19.53%, lr=['1e-05'], val_acc: 23.63%
# iter_one_train_time: 0.9713468551635742 seconds, last one_val_time: 17.197083711624146 seconds

# train: 1-148/391 tr_acc: 17.19%, lr=['1e-05'], val_acc: 24.43%:  38%|███▊      | 149/391 [03:49<03:38,  1.11it/s]
# 1-149 training acc: 19.53%, lr=['1e-05'], val_acc: 24.43%
# iter_one_train_time: 0.8979551792144775 seconds, last one_val_time: 16.401078462600708 seconds

# train: 1-198/391 tr_acc: 28.91%, lr=['1e-05'], val_acc: 25.07%:  51%|█████     | 199/391 [04:51<02:52,  1.12it/s]
# 1-199 training acc: 21.09%, lr=['1e-05'], val_acc: 25.07%
# iter_one_train_time: 0.886080265045166 seconds, last one_val_time: 17.012481927871704 seconds

# train: 1-248/391 tr_acc: 24.22%, lr=['1e-05'], val_acc: 25.92%:  64%|██████▎   | 249/391 [05:59<03:39,  1.55s/it]
# 1-249 training acc: 22.66%, lr=['1e-05'], val_acc: 25.92%
# iter_one_train_time: 1.5489864349365234 seconds, last one_val_time: 16.566229104995728 seconds



### conv fc weight gradient 방향바꾸는거

정상

In [ ]:
# ### my_snn control board ########################
# decay = 0.5 # 0.875 0.25 0.125 0.75 0.5

# my_snn_system(  devices = "0,1,2,3,4,5",
#                 my_seed = 42,
#                 TIME = 8, # dvscifar 10
#                 BATCH = 128, # batch norm 할거면 2이상으로 해야함
#                 IMAGE_SIZE = 32, # dvscifar 48 # MNIST 28 # CIFAR10 32
#                 which_data = 'CIFAR10',# 'CIFAR10' 'MNIST' 'FASHION_MNIST' 'DVS-CIFAR10'
#                 CLASS_NUM = 10,
#                 data_path = '/data2', # YOU NEED TO CHANGE THIS
#                 rate_coding = False, # True # False

#                 lif_layer_v_init = 0.0,
#                 lif_layer_v_decay = decay,
#                 lif_layer_v_threshold = 1.0,
#                 lif_layer_v_reset = 0.0, #현재 안씀. 걍 빼기 해버림
#                 lif_layer_sg_width = 1.0, # surrogate sigmoid 쓸 때는 의미없음

#                 # synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
#                 synapse_conv_kernel_size = 3,
#                 synapse_conv_stride = 1,
#                 synapse_conv_padding = 1,
#                 synapse_conv_trace_const1 = 1,
#                 synapse_conv_trace_const2 = decay, # lif_layer_v_decay

#                 # synapse_fc_out_features = CLASS_NUM,
#                 synapse_fc_trace_const1 = 1,
#                 synapse_fc_trace_const2 = decay, # lif_layer_v_decay

#                 pre_trained = False, # True # False
#                 convTrue_fcFalse = True, # True # False

#                 # 'P' for average pooling, 'M' for maxpooling, 'L' for linear classifier, [  ] for residual block
#                 # cfg = [64],
#                 # cfg = [64,[64,64],64], # 끝에 linear classifier 하나 자동으로 붙습니다
#                 cfg = [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
#                 # cfg = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512],
#                 # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512],
#                 # cfg = [64, 'P', 128, 'P', 256, 256, 'P', 512, 512, 512, 512, 'L', 4096, 4096],


#                 net_print = False, # True # False
#                 pre_trained_path = "net_save/save_now_net.pth",
#                 learning_rate = 0.001,
#                 epoch_num = 200,
#                 verbose_interval = 50, #숫자 크게 하면 꺼짐
#                 validation_interval = 50, #숫자 크게 하면 꺼짐
#                 tdBN_on = False,  # True # False
#                 BN_on = True,  # True # False
                
#                 surrogate = 'sigmoid', # 'rectangle' 'sigmoid' 'rough_rectangle'
                
#                 gradient_verbose = False,  # True # False  # weight gradient 각 layer마다 띄워줌

#                 BPTT_on = False,  # True # False # True이면 BPTT, False이면 OTTT

#                 scheduler_name = 'no', # 'no' 'StepLR' 'ExponentialLR' 'ReduceLROnPlateau' 'CosineAnnealingLR' 'OneCycleLR'
                
#                 ddp_on = False,
#                 )
# # sigmoid와 BN이 있어야 잘된다.
# # average pooling이 낫다.

# # nda에서는 decay = 0.25, threshold = 0.5, width =1, surrogate = rectangle
# ## OTTT 에서는 decay = 0.5, threshold = 1.0, surrogate = sigmoid

# Files already downloaded and verified
# Files already downloaded and verified
# ==================================================
# My Num of PARAMS: 9,302,410, system's param_num : 9,346,442
# Memory: 35.49MiB at 32-bit
# # 
# EPOCH 0
# train: 0-48/391 tr_acc: 22.66%, lr=['0.001'], tr_loss: 290.5330505371094, val_acc: 0.00%:  13%|█▎        | 49/391 [00:59<05:11,  1.10it/s] 
# 0-49 training acc: 21.09%, lr=['0.001'], val_acc: 0.00%
# iter_one_train_time: 0.8913624286651611 seconds, last one_val_time: 0 seconds

# train: 0-98/391 tr_acc: 24.22%, lr=['0.001'], tr_loss: 228.34286499023438, val_acc: 24.43%:  25%|██▌       | 99/391 [02:02<04:22,  1.11it/s]
# 0-99 training acc: 17.97%, lr=['0.001'], val_acc: 24.43%
# iter_one_train_time: 0.9158523082733154 seconds, last one_val_time: 16.69316840171814 seconds

# train: 0-148/391 tr_acc: 32.03%, lr=['0.001'], tr_loss: 210.59719848632812, val_acc: 27.37%:  38%|███▊      | 149/391 [03:04<03:40,  1.10it/s]
# 0-149 training acc: 28.91%, lr=['0.001'], val_acc: 27.37%
# iter_one_train_time: 0.9076290130615234 seconds, last one_val_time: 16.61735248565674 seconds

# train: 0-198/391 tr_acc: 35.16%, lr=['0.001'], tr_loss: 195.4486846923828, val_acc: 26.79%:  51%|█████     | 199/391 [04:06<02:57,  1.08it/s] 
# 0-199 training acc: 30.47%, lr=['0.001'], val_acc: 26.79%
# iter_one_train_time: 0.8877060413360596 seconds, last one_val_time: 16.78444528579712 seconds

# train: 0-248/391 tr_acc: 25.00%, lr=['0.001'], tr_loss: 209.00848388671875, val_acc: 30.89%:  64%|██████▎   | 249/391 [05:08<02:10,  1.09it/s]
# 0-249 training acc: 27.34%, lr=['0.001'], val_acc: 30.89%
# iter_one_train_time: 0.8717858791351318 seconds, last one_val_time: 16.502493143081665 seconds

# train: 0-298/391 tr_acc: 30.47%, lr=['0.001'], tr_loss: 220.2567138671875, val_acc: 31.96%:  76%|███████▋  | 299/391 [06:11<01:23,  1.10it/s] 
# 0-299 training acc: 29.69%, lr=['0.001'], val_acc: 31.96%
# iter_one_train_time: 0.9970495700836182 seconds, last one_val_time: 16.796446800231934 seconds



conv만 이상하게

In [ ]:
# Files already downloaded and verified
# Files already downloaded and verified
# ==================================================
# My Num of PARAMS: 9,302,410, system's param_num : 9,346,442
# Memory: 35.49MiB at 32-bit
# ==================================================
# EPOCH 0
# train: 0-48/391 tr_acc: 8.59%, lr=['0.001'], tr_loss: 1039.622802734375, val_acc: 0.00%:  13%|█▎        | 49/391 [00:59<04:57,  1.15it/s]  
# 0-49 training acc: 6.25%, lr=['0.001'], val_acc: 0.00%
# iter_one_train_time: 0.8593673706054688 seconds, last one_val_time: 0 seconds

# train: 0-98/391 tr_acc: 7.81%, lr=['0.001'], tr_loss: 339.25616455078125, val_acc: 7.53%:  25%|██▌       | 99/391 [02:01<04:22,  1.11it/s] 
# 0-99 training acc: 5.47%, lr=['0.001'], val_acc: 7.53%
# iter_one_train_time: 0.8696670532226562 seconds, last one_val_time: 17.034608364105225 seconds

# train: 0-148/391 tr_acc: 4.69%, lr=['0.001'], tr_loss: 80.27715301513672, val_acc: 6.74%:  38%|███▊      | 149/391 [03:02<03:39,  1.10it/s]  
# 0-149 training acc: 7.81%, lr=['0.001'], val_acc: 6.74%
# iter_one_train_time: 0.8959760665893555 seconds, last one_val_time: 16.716905117034912 seconds

# train: 0-198/391 tr_acc: 9.38%, lr=['0.001'], tr_loss: 41.17329788208008, val_acc: 7.03%:  51%|█████     | 199/391 [04:04<02:53,  1.10it/s]  
# 0-199 training acc: 9.38%, lr=['0.001'], val_acc: 7.03%
# iter_one_train_time: 0.8900871276855469 seconds, last one_val_time: 17.06007432937622 seconds

# train: 0-217/391 tr_acc: 7.81%, lr=['0.001'], tr_loss: 38.29032516479492, val_acc: 8.52%:  56%|█████▌    | 218/391 [04:39<02

conv fc 둘다 이상하게

In [ ]:
# Files already downloaded and verified
# Files already downloaded and verified
# ==================================================
# My Num of PARAMS: 9,302,410, system's param_num : 9,346,442
# Memory: 35.49MiB at 32-bit
# ==================================================
# EPOCH 0
# train: 0-48/391 tr_acc: 7.03%, lr=['0.001'], tr_loss: 603.5956420898438, val_acc: 0.00%:  13%|█▎        | 49/391 [00:59<05:09,  1.10it/s]  
# 0-49 training acc: 8.59%, lr=['0.001'], val_acc: 0.00%
# iter_one_train_time: 0.8527345657348633 seconds, last one_val_time: 0 seconds

# train: 0-98/391 tr_acc: 5.47%, lr=['0.001'], tr_loss: 75.4881362915039, val_acc: 10.00%:  25%|██▌       | 99/391 [02:01<04:21,  1.12it/s]   
# 0-99 training acc: 14.06%, lr=['0.001'], val_acc: 10.00%
# iter_one_train_time: 0.8860585689544678 seconds, last one_val_time: 16.647587537765503 seconds

# train: 0-148/391 tr_acc: 8.59%, lr=['0.001'], tr_loss: 77.6851577758789, val_acc: 10.00%:  38%|███▊      | 149/391 [03:03<03:37,  1.11it/s]   
# 0-149 training acc: 11.72%, lr=['0.001'], val_acc: 10.00%
# iter_one_train_time: 0.8722302913665771 seconds, last one_val_time: 16.754177808761597 seconds

# train: 0-157/391 tr_acc: 12.50%, lr=['0.001'], tr_loss: 84.20757293701172, val_acc: 10.00%:  40%|████      | 158/391 [03:28<04:36,  1.18s/it]

fc만 이상하게

In [ ]:
# Files already downloaded and verified
# Files already downloaded and verified
# ==================================================
# My Num of PARAMS: 9,302,410, system's param_num : 9,346,442
# Memory: 35.49MiB at 32-bit
# ==================================================
# EPOCH 0
# train: 0-48/391 tr_acc: 22.66%, lr=['0.001'], tr_loss: 341.66009521484375, val_acc: 0.00%:  13%|█▎        | 49/391 [00:59<05:13,  1.09it/s]
# 0-49 training acc: 21.88%, lr=['0.001'], val_acc: 0.00%
# iter_one_train_time: 0.870579719543457 seconds, last one_val_time: 0 seconds

# train: 0-98/391 tr_acc: 27.34%, lr=['0.001'], tr_loss: 248.8526153564453, val_acc: 24.19%:  25%|██▌       | 99/391 [02:00<04:21,  1.11it/s] 
# 0-99 training acc: 30.47%, lr=['0.001'], val_acc: 24.19%
# iter_one_train_time: 0.8846514225006104 seconds, last one_val_time: 16.405277729034424 seconds

# train: 0-148/391 tr_acc: 29.69%, lr=['0.001'], tr_loss: 226.84280395507812, val_acc: 25.52%:  38%|███▊      | 149/391 [03:03<03:39,  1.10it/s]
# 0-149 training acc: 25.00%, lr=['0.001'], val_acc: 25.52%
# iter_one_train_time: 0.8858129978179932 seconds, last one_val_time: 16.496455907821655 seconds

# train: 0-198/391 tr_acc: 33.59%, lr=['0.001'], tr_loss: 160.5304718017578, val_acc: 28.07%:  51%|█████     | 199/391 [04:05<02:49,  1.13it/s] 
# 0-199 training acc: 24.22%, lr=['0.001'], val_acc: 28.07%
# iter_one_train_time: 0.9705793857574463 seconds, last one_val_time: 16.123810291290283 seconds

# train: 0-248/391 tr_acc: 33.59%, lr=['0.001'], tr_loss: 206.28160095214844, val_acc: 28.51%:  64%|██████▎   | 249/391 [05:07<02:12,  1.07it/s]
# 0-249 training acc: 32.81%, lr=['0.001'], val_acc: 28.51%
# iter_one_train_time: 0.8938279151916504 seconds, last one_val_time: 16.548280239105225 seconds

# train: 0-298/391 tr_acc: 32.03%, lr=['0.001'], tr_loss: 214.91668701171875, val_acc: 30.11%:  76%|███████▋  | 299/391 [06:10<01:26,  1.07it/s]
# 0-299 training acc: 32.03%, lr=['0.001'], val_acc: 30.11%
# iter_one_train_time: 0.8992974758148193 seconds, last one_val_time: 16.498471975326538 seconds

# train: 0-348/391 tr_acc: 37.50%, lr=['0.001'], tr_loss: 199.61183166503906, val_acc: 29.93%:  89%|████████▉ | 349/391 [07:11<00:37,  1.11it/s]
# 0-349 training acc: 37.50%, lr=['0.001'], val_acc: 29.93%
# iter_one_train_time: 0.9911439418792725 seconds, last one_val_time: 16.14445948600769 seconds

# train: 0-390/391 tr_acc: 47.50%, lr=['0.001'], tr_loss: 126.8869400024414, val_acc: 34.47%: 100%|██████████| 391/391 [08:06<00:00,  1.24s/it] 
# epoch_time: 486.4687490463257 seconds
